IMPORTING REQUIRED PACKAGES

In [2]:
from packages_imp import*
np.random.seed(34)

#Importing functions from jupyter notebook
from ipynb.fs.full.dat_preprocess import*


In [3]:
# Calling gpu based torch model
import torch
torch.manual_seed(1) # No of instances
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

DECLARE THE VARIABLES TO BE USED

In [4]:
removed_columns=[]  # Stores the list of columns to be dropped

LOAD THE FINAL DATASET AFTER EDA

In [5]:
def call_EDA(file):
  filename=file
  res = pm.execute_notebook(
    'EDA_and_Feature_Selection.ipynb'  ,
    'CNN_LSTM_training_model.ipynb',
    parameters = dict(filename=filename),
    log_output=True,
    stdout_file=sys.stdout,
    stderr_file=sys.stderr,
    )
call_EDA('file1.csv')

Executing:   0%|          | 0/20 [00:00<?, ?cell/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ENGINE_ID        20631 non-null  int64  
 1   CONDITION_ID     20631 non-null  int64  
 2   Cycle_Time       20631 non-null  int64  
 3   OpSet1           20631 non-null  float64
 4   OpSet2           20631 non-null  float64
 5   OpSet3           20631 non-null  float64
 6   SensorMeasure1   20631 non-null  float64
 7   SensorMeasure2   20631 non-null  float64
 8   SensorMeasure3   20631 non-null  float64
 9   SensorMeasure4   20631 non-null  float64
 10  SensorMeasure5   20631 non-null  float64
 11  SensorMeasure6   20631 non-null  float64
 12  SensorMeasure7   20631 non-null  float64
 13  SensorMeasure8   20631 non-null  float64
 14  SensorMeasure9   20631 non-null  float64
 15  SensorMeasure10  20631 non-null  float64
 16  SensorMeasure11  20631 non-null  float64
 17  SensorMeasur

In [6]:
#Reading the list of columns to be dropped from training and test files from removed_columns.data
import pickle

with open('removed_columns.data', 'rb') as filehandle:
    # Read the data as a binary data stream
    removed_columns = pickle.load(filehandle)
removed_columns

['ENGINE_ID',
 'OpSet1',
 'OpSet2',
 'CONDITION_ID',
 'OpSet3',
 'SensorMeasure1',
 'SensorMeasure5',
 'SensorMeasure6',
 'SensorMeasure10',
 'SensorMeasure16',
 'SensorMeasure18',
 'SensorMeasure19']

In [7]:
#Delete columns from removed_columns which are not required

In [10]:
train_data=pd.read_csv('file1.csv')
train_data=train_data.drop(columns=removed_columns)
training_columns=train_data.columns
training_columns

Index(['Cycle_Time', 'SensorMeasure2', 'SensorMeasure3', 'SensorMeasure4',
       'SensorMeasure7', 'SensorMeasure8', 'SensorMeasure9', 'SensorMeasure11',
       'SensorMeasure12', 'SensorMeasure13', 'SensorMeasure14',
       'SensorMeasure15', 'SensorMeasure17', 'SensorMeasure20',
       'SensorMeasure21', 'RUL'],
      dtype='object')

In [9]:
#WINDOWING
WINDOW_SIZE=30
BATCH_SIZE = 210   # No of samples processed before model is updated


In [11]:
#Process target RUL 
def process_target_RUL(data_length, early_rul = None):
    """ 
    Takes datalength and earlyrul as input and 
    creates target rul.
    """
    if early_rul == None:
        return np.arange(data_length-1, -1, -1)
    else:
        early_rul_duration = data_length - early_rul
        if early_rul_duration <= 0:   # 
            return np.arange(data_length-1, -1, -1)
        else:
            return np.append(early_rul*np.ones(shape = (early_rul_duration,)), np.arange(early_rul-1, -1, -1))

In [13]:
# Creating a numpy array with tuple containing feature and labels
def process_dataset_with_target_RUL(input_data, target_data = None, window_length = 1, shift = 1):
    """
    Description:
    
    Depending on values of window_length and shift, this function generates batchs of data and targets 
    from input_data and target_data.
    
    Number of batches taken as a function of engine ID = np.floor((len(input_data) - window_length)/shift) + 1
    
    **Input dimensions have not been checked and considered to be at par with the requirements**
    
    Arguments:
        input_data: input data to function (2D array containing features and label)
        target_data: input RUL values (1D array)
        window_length: window length of data
        shift: Distance by which the window moves for next batch. This is closely related to overlap
               between data. For example, if window length is 30 and shift is 1, there is an overlap of 
               29 data points between two consecutive batches.
        
    """
    num_batches = np.int(np.floor((len(input_data) - window_length)/shift)) + 1
    num_features = input_data.shape[1]
    output_data = np.repeat(np.nan, repeats = num_batches * window_length * num_features).reshape(num_batches, window_length,
                                                                                                  num_features)
    if target_data is None:
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
        return output_data
    else:
        output_targets = np.repeat(np.nan, repeats = num_batches)
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
            output_targets[batch] = target_data[(shift*batch + (window_length-1))]
        return output_data, output_targets